# Cleaning and Profiling

In [1]:
""" Notebook Setup/Imports
"""

# Project path setup
from pathlib import Path
project_dir = Path(__name__).resolve().parents[1]

# Library import
import pandas as pd
import numpy as np
import pandas_profiling

# Print strings as markdown
from IPython.display import Markdown

# Data variables from previous notebook
dataset = pd.read_pickle(Path.joinpath(project_dir, "data/processed/0_data.pickle"))
data_dictionary = pd.read_pickle(Path.joinpath(project_dir, "data/processed/0_data_dictionary.pickle"))

## Data Type Validation & Cleaning

In [2]:
""" Get the count for data types in the dataset
"""
d = dataset.dtypes
dataset.get_dtype_counts()

bool         4
float64    314
object       9
dtype: int64

In [3]:
""" Find columns as strings or other objects
"""
object_columns = d[d == 'object'].index.to_list()
print(object_columns)

['2010 Census Population', 'Population Estimate, 2011', 'Population Estimate, 2012', 'Population Estimate, 2013', 'Population Estimate, 2014', 'Population Estimate, 2015', 'Population Estimate, 2016', 'School Breakfast Program participants FY 2011', 'School Breakfast Program participants, FY 2012']


In [4]:
""" Parse objects to float
"""
for column in object_columns:
    sub = dataset[column].str.replace(",","").str.replace("-","").str.strip().apply(lambda x: np.nan if x == "" else x)
    dataset[column] = sub.astype(float)

In [5]:
""" Check types
"""
dataset.get_dtype_counts()

bool         4
float64    323
dtype: int64

## Variable Selection

In [6]:
""" Create a list of rejected variables with > .975 correlation from the profile analysis
"""
profile = pandas_profiling.ProfileReport(dataset, check_recoded=True)
rejected_variables = profile.get_rejected_variables(.975)

In [7]:
""" List of columns to keep
"""
keep_columns = [x for x in dataset.columns if x not in rejected_variables]

In [8]:
""" New dataframe with the columns to keep
"""
dataset_clean = dataset[keep_columns]

In [9]:
""" Set the USDA Model to the percentage 
"""
dataset_clean.drop("USDA Model Count", axis="columns", inplace=True)

/anaconda3/envs/DS420/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
dataset_clean.rename({"USDA Model Percent":"USDA Model"},axis="columns",inplace=True)

/anaconda3/envs/DS420/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [11]:
""" Save the cleaned data
"""
dataset_clean.to_pickle(Path.joinpath(project_dir, "data/processed/1_data.pickle"))
data_dictionary.to_pickle(Path.joinpath(project_dir, "data/processed/1_data_dictionary.pickle"))

## Cleaned Data Profile

In [12]:
""" Create the data profile object and save it to an HTML file
"""
profile = pandas_profiling.ProfileReport(dataset_clean, check_recoded=True)
profile.to_file(outputfile = Path.joinpath(project_dir,
                                      "docs/_static/profile.html"))